# Лабораторная работа №13 по PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Загрузка датасета MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_loader = DataLoader(datasets.MNIST("data", train=True, download=True, transform=transform), batch_size=64, shuffle=True)

In [3]:
# Создание класса для перцептрона
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(28*28, 10)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc(x.view(x.size(0), -1)))
        return x

In [4]:
# Инициализация модели, функции потерь и оптимизатора
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Perceptron().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# Обучение модели
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item()}')

Epoch 1, Batch 0, Loss: 2.3215129375457764
Epoch 1, Batch 100, Loss: 1.8955930471420288
Epoch 1, Batch 200, Loss: 1.8103258609771729
Epoch 1, Batch 300, Loss: 1.7851431369781494
Epoch 1, Batch 400, Loss: 1.7442611455917358
Epoch 1, Batch 500, Loss: 1.7117027044296265
Epoch 1, Batch 600, Loss: 1.748612880706787
Epoch 1, Batch 700, Loss: 1.7003720998764038
Epoch 1, Batch 800, Loss: 1.7485405206680298
Epoch 1, Batch 900, Loss: 1.6677813529968262
Epoch 2, Batch 0, Loss: 1.7153115272521973
Epoch 2, Batch 100, Loss: 1.7048536539077759
Epoch 2, Batch 200, Loss: 1.6981478929519653
Epoch 2, Batch 300, Loss: 1.6641005277633667
Epoch 2, Batch 400, Loss: 1.6438183784484863
Epoch 2, Batch 500, Loss: 1.6900931596755981
Epoch 2, Batch 600, Loss: 1.646651029586792
Epoch 2, Batch 700, Loss: 1.684612512588501
Epoch 2, Batch 800, Loss: 1.6305265426635742
Epoch 2, Batch 900, Loss: 1.6537400484085083
Epoch 3, Batch 0, Loss: 1.669937014579773
Epoch 3, Batch 100, Loss: 1.6217759847640991
Epoch 3, Batch 200, 

In [6]:
# Проверка обученной модели
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Точность перцептрона на обучающей выборке: {100 * accuracy:.2f}%')

Точность перцептрона на обучающей выборке: 89.10%
